# Virtual Concierge 

This notebook demonstrates face recogntion with using the [InsightFace](https://github.com/deepinsight/insightface) model on MXNET.

### Prerequisites

The following packages need to be installed before proceeding:

* MXNet - `pip install mxnet`
* numpy - `1pip install numpy`
* OpenCV - `pip install opencv-python`
* Graphviz - `pip install graphviz`
* matplotlib - `pip install matplotlib`
* Seaborn `sudo pip3 install seaborn`
* Boto3 - `pip install boto3`

### Import dependencies

Verify that all dependencies are installed using the cell below. Continue if no errors encountered, warnings can be ignored.

In [ ]:
import cv2
import sys
import numpy as np
import mxnet as mx
import os
from __future__ import print_function

from matplotlib import pyplot as plt
%matplotlib inline

import boto3
import json

### Load pretrained model

`get_model()` : Loads ONNX model into MXNet symbols and params, defines model using symbol file and binds parameters to the model using params file.

In [ ]:
def get_model(ctx, image_size, model_str, layer):
    _vec = model_str.split(',')
    assert len(_vec)==2
    prefix = _vec[0]
    epoch = int(_vec[1])
    print('loading',prefix, epoch)
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    all_layers = sym.get_internals()
    sym = all_layers[layer+'_output']
    model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)
    model.bind(data_shapes=[('data', (1, 3, image_size[0], image_size[1]))])
    model.set_params(arg_params, aux_params)
    return model, sym

### Preprocess images

In order to input only face pixels into the network, all input images are passed through a pretrained face detection and alignment model as described above. The output of this model are landmark points and a bounding box corresponding to the face in the image. Using this output, the image is processed using affine transforms to generate the aligned face images which are input to the network. The functions performing this is defined below.

`get_input()` : Returns aligned face to the bbox and margin

`show_input()` : Shows the image after transposing it

In [ ]:
def get_input(img, image_size, bbox=None, margin=44):
    if bbox is None:
        det = np.zeros(4, dtype=np.int32)
        det[0] = int(img.shape[1]*0.0625)
        det[1] = int(img.shape[0]*0.0625)
        det[2] = img.shape[1] - det[0]
        det[3] = img.shape[0] - det[1]
    else:
        det = bbox
    bb = np.zeros(4, dtype=np.int32)
    bb[0] = np.maximum(det[0]-margin/2, 0)
    bb[1] = np.maximum(det[1]-margin/2, 0)
    bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
    bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
    img = img[bb[1]:bb[3],bb[0]:bb[2],:]
    img = cv2.resize(img, (image_size[1], image_size[0]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aligned = np.transpose(img, (2,0,1))
    return aligned

def show_input(aligned):
    plt.imshow(np.transpose(aligned,(1,2,0)))

### Get Features

`l2_normalize()`: Performs row normalization on the vector

`get_feature()` : Performs forward pass on the data aligned using model and returns the embedding

In [ ]:
def l2_normalize(X):
    norms = np.sqrt((X * X).sum(axis=1))
    X /= norms[:, np.newaxis]
    return X

def get_feature(model, aligned):
    input_blob = np.expand_dims(aligned, axis=0)
    data = mx.nd.array(input_blob)
    db = mx.io.DataBatch(data=(data,))
    model.forward(db, is_train=False)
    embedding = model.get_outputs()[0].asnumpy()
    embedding = l2_normalize(embedding).flatten()
    return embedding

### Visualize Model

Load the model on the cpu, then compare badge image images to an aligned image

In [ ]:
%%time

image_size = (112,112)
model_name = './models/mobilenet1,0'
model, sym = get_model(mx.cpu(), image_size, model_name, 'fc1')

In [ ]:
# Plot the model
mx.viz.plot_network(sym)

### Evaluation

Load test badge image and compare to uploaded image

In [ ]:
img1 = cv2.imread('./people/julbrigh.jpg')
pre1 = get_input(img1, image_size)
show_input(pre1)

In [ ]:
# img1 = cv2.imread('./people/sssalim.jpg')
# pre1 = get_input(img1, image_size)
# show_input(pre1)

In [ ]:
img2 = cv2.imread('./people/julbrigh_deeplens.jpg')
pre2 = get_input(img2, image_size, bbox=[1368,376,1573,632])
show_input(pre2)

In [ ]:
# img2 = cv2.imread('./people/sssalim_deeplens.jpg')
# pre2 = get_input(img2, image_size, bbox=[1090,389,1320,663])
# show_input(pre2)

### Generate predictions

Two face images are passed through the network sequentially to generate embedding vectors for each. 

In [ ]:
# given the model 
out1 = get_feature(model,pre1)
out2 = get_feature(model,pre2)
out1[:3], out2[:3]

### Calculate similarity

The squared distance and cosine similarity between the embedding vectors are computed and displayed. Images containing face of a single person will have low distance and high similarity and vice-versa. 

The distance values are in [0,4) and similarity values in [-1,1].

In [ ]:
# Compute squared distance between embeddings
dist = np.sum(np.square(out2-out1))
# Compute cosine similarity between embedddings
sim = np.dot(out1, out2.T)
# Print predictions
print('Distance = %f' %(dist))
print('Similarity = %f' %(sim))

### Plot Distribution

Load the saved vectors for all people in the database, and plot the distribution and outliner for match.

In [ ]:
%%time

from scipy import stats
import seaborn as sns

# Load vectors and names
people = np.load('./models/people-au.npz')
vecs = people['vecs']
names = [p.decode('utf-8') for p in people['names']]

# calculate cosine similarity and relative zscores
sims = np.dot(vecs, out2)
zscores = stats.zscore(sims)

# plot series and print score and name
sns.set(color_codes=True)
plt.subplots(figsize=(10,6))
ax = sns.distplot(zscores, bins=50, kde=False, rug=True)
ax.set(xlabel='zscore', ylabel='number of people')
plt.title('zscore distribution')
plt.show()

In [ ]:
idx = sims.argmax()
print('sim: {}, zscore: {}, name: {}'.format(sims[idx], zscores[idx], names[idx]))